In [1]:
!nvidia-smi

Sat Sep 14 17:42:32 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import tensorflow as tf
tf.enable_eager_execution()

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam

In [3]:
!git clone https://github.com/akshitbhalla/NeuralDot.git
!python /content/NeuralDot/createTFRecords.py

fatal: destination path 'NeuralDot' already exists and is not an empty directory.
Download from https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz and extract.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please write your own downloading logic.
Generating train.tfrecords


Generating validation.tfrecords
Generating eval.tfrecords
Done!


In [0]:
filename_train = 'train.tfrecords' 
filename_test = 'validation.tfrecords' 
img_height = 32
img_width = 32
channels = 3

In [0]:
def de_serialize(serialized_example):
    features = tf.parse_single_example(
    serialized_example,
    features={
    'image': tf.FixedLenFeature([], tf.string),
    'label': tf.FixedLenFeature([], tf.int64),
    })
    image = tf.decode_raw(features['image'], tf.uint8)
    image.set_shape([channels * img_height * img_width])
    image = tf.cast(
    tf.transpose(tf.reshape(image, [channels, img_height, img_width]), [1, 2, 0]),
    tf.float32)
    label = tf.cast(features['label'], tf.int32)
    label = tf.one_hot(label, 10)
    return image, label

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 100
num_filter = 20
compression = 0.5
dropout_rate = 0.15
l = 10

In [0]:
# -----------TRAIN TFRecords
dataset_train = tf.data.TFRecordDataset(filename_train).repeat(epochs)
dataset_train = dataset_train.map(de_serialize)
dataset_train = dataset_train.batch(batch_size)

# -------------TEST TFRecords
dataset_test = tf.data.TFRecordDataset(filename_test).repeat(1)
dataset_test = dataset_test.map(de_serialize)
dataset_test = dataset_test.batch(batch_size)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
input = Input(shape=(img_height, img_width, channels,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [12]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 20)   540         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 20)   80          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 20)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [14]:
for zz in range(1,epochs+1):
  # determine Loss function and Optimizer
  print("\nEpoch {}/{}".format(zz,epochs))
  model.fit_generator(dataset_train,steps_per_epoch=624,epochs=1,verbose=1)

  model.evaluate_generator(dataset_test,steps=156,verbose=1)


Epoch 1/100
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
156/156 [==============================] - 5s 33ms/step - loss: 1.3442 - acc: 0.5048

Epoch 2/100
156/156 [==============================] - 4s 23ms/step - loss: 1.3564 - acc: 0.5531

Epoch 3/100
156/156 [==============================] - 4s 23ms/step - loss: 1.4503 - acc: 0.5510

Epoch 4/100
156/156 [==============================] - 4s 22ms/step - loss: 1.0204 - acc: 0.6594

Epoch 5/100
156/156 [==============================] - 4s 22ms/step - loss: 0.8903 - acc: 0.6926

Epoch 6/100
156/156 [==============================] - 3s 22ms/step - loss: 0.9415 - acc: 0.6870

Epoch 7/100
156/156 [==============================] - 3s 22ms/step - loss: 0.9645 - acc: 0.6850

Epoch 8/100
156/156 [==============================] - 3s 22ms/step - loss: 0.7338 - acc: 0.7518

Epoch 9/100
156/156 [==============================] - 4s 23ms/step - loss: 0.9629 - acc: 0.6971

Epoch 10/100
156/156 [=

In [16]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk
